# Part 1: Build CpG Detector

Here we have a simple problem, given a DNA sequence (of N, A, C, G, T), count the number of CpGs in the sequence (consecutive CGs).

We have defined a few helper functions / parameters for performing this task.

We need you to build a LSTM model and train it to complish this task in PyTorch.

A good solution will be a model that can be trained, with high confidence in correctness.

In [282]:
import random
import torch
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
from typing import Sequence
from functools import partial
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error as MAE
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

## Data Preparation

In [283]:
# DO NOT CHANGE HERE
def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(13)

# Use this for getting x label
def rand_sequence(n_seqs: int, seq_len: int=128) -> Sequence[int]:
    for i in range(n_seqs):
        yield [random.randint(0, 4) for _ in range(seq_len)]

# Use this for getting y label
def count_cpgs(seq: str) -> int:
    cgs = 0
    for i in range(0, len(seq) - 1):
        dimer = seq[i:i+2]
        # note that seq is a string, not a list
        if dimer == "CG":
            cgs += 1
    return cgs

# Alphabet helpers   
alphabet = 'NACGT'
dna2int = { a: i for a, i in zip(alphabet, range(5))}
int2dna = { i: a for a, i in zip(alphabet, range(5))}

intseq_to_dnaseq = partial(map, int2dna.get)
dnaseq_to_intseq = partial(map, dna2int.get)

In [284]:
# we prepared two datasets for training and evaluation
# training data scale we set to 2048
# we test on 512
set_seed(13)
def prepare_data(num_samples=100):
    # prepared the training and test data
    # you need to call rand_sequence and count_cpgs here to create the dataset
    # step 1
    X_dna_seqs_train = list(rand_sequence(num_samples))
    """
    hint:
        1. You can check X_dna_seqs_train by print, the data is ids which is your training X 
        2. You first convert ids back to DNA sequence
        3. Then you run count_cpgs which will yield CGs counts - this will be the labels (Y)
    """
    #step2 : # use intseq_to_dnaseq here to convert ids back to DNA seqs 
    temp = [list(intseq_to_dnaseq(i)) for i in X_dna_seqs_train] 
    
    #step3 : # use count_cpgs here to generate labels with temp generated in step2 
    y_dna_seqs = [count_cpgs(''.join(i)) for i in temp]
    return X_dna_seqs_train, y_dna_seqs
    
train_x, train_y = prepare_data(2048)
test_x, test_y = prepare_data(512)

## Data Set Class for Data Loader

In [285]:
# create data loader
class DNASeqData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

train_dataset = DNASeqData(torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float())
test_dataset = DNASeqData(torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float())  

## LSTM Network 

In [286]:
# MODEL ARCHITECTURE 
class CpGDetector(nn.Module):

    def __init__(self, N_CLASSES, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS,N_EMBEDDINGS):
        super(CpGDetector, self).__init__()

        self.embedding = nn.Embedding(INPUT_SIZE,N_EMBEDDINGS)
        self.num_classes = N_CLASSES
        self.num_layers = N_LAYERS
        self.hidden_size = HIDDEN_SIZE

        self.lstm = nn.LSTM(
            input_size=N_EMBEDDINGS, 
            hidden_size=HIDDEN_SIZE,
            num_layers=N_LAYERS
        )
        self.drop = nn.Dropout(p=0.5)
        self.fc = nn.Linear(HIDDEN_SIZE,self.num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.int()
        embedded = self.embedding(x)  # adding embedding layer 
        output, _ = self.lstm(embedded) # passing the embedding vector through the lstm unit 
        output = self.drop(output[:, -1, :]) # taking the last hidden state after dropout 
        output = self.fc(self.relu(output))  # activation layer 
        return output.T

## Model Training 

In [287]:
MODEL_PARAMETERS = {
    'N_CLASSES':1,
    'INPUT_SIZE':128,
    'HIDDEN_SIZE':64,
    'N_LAYERS':1,
    'N_EMBEDDINGS':32
}

In [288]:
set_seed(13)
MODEL = CpGDetector(**MODEL_PARAMETERS)
TRAINING_CONFIG = {
    'N_EPOCHS':25,
    'BATCH_SIZE':8,
    'LEARNING_RATE':0.01,
    'CRITERION':torch.nn.MSELoss()
}
OPTIMIZER = torch.optim.Adam(MODEL.parameters(), lr=TRAINING_CONFIG['LEARNING_RATE'])

In [289]:
# creatin the torch data loader class for train and test
set_seed(13)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=TRAINING_CONFIG['BATCH_SIZE'], shuffle=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=TRAINING_CONFIG['BATCH_SIZE'])

In [290]:
# Train the model
set_seed(13)
for epoch in range(TRAINING_CONFIG['N_EPOCHS']):
    EPOCH_LOSSES = 0 
    for j,(x_train,y_train) in enumerate(train_data_loader):
        outputs = MODEL(x_train)
        OPTIMIZER.zero_grad()
        y_train = y_train.unsqueeze(0)
        loss = TRAINING_CONFIG['CRITERION'](outputs, y_train)
        test_loss_total = 0 
        with torch.inference_mode():
            test_loss_total = 0 
            for j,(x_test,y_test) in enumerate(test_data_loader):
                test_output = MODEL(x_test)
                y_test = y_test.unsqueeze(0)
                test_loss = TRAINING_CONFIG['CRITERION'](test_output,y_test)
                test_loss_total+=test_loss.item()
                
        EPOCH_LOSSES+=loss.item()
        loss.backward()
        OPTIMIZER.step()
    if epoch % 1 == 0:
        print("Epoch: %d, Train loss: %1.5f Test loss : %1.5f"  % (epoch, EPOCH_LOSSES,test_loss_total)) 

Epoch: 0, Train loss: 1496.97855 Test loss : 323.32062
Epoch: 1, Train loss: 1263.74828 Test loss : 315.11848
Epoch: 2, Train loss: 1275.31169 Test loss : 306.82204
Epoch: 3, Train loss: 1232.96073 Test loss : 306.41611
Epoch: 4, Train loss: 1227.91676 Test loss : 307.04581
Epoch: 5, Train loss: 1211.53485 Test loss : 297.24400
Epoch: 6, Train loss: 1190.54235 Test loss : 282.44889
Epoch: 7, Train loss: 1166.51572 Test loss : 283.83205
Epoch: 8, Train loss: 1144.05296 Test loss : 293.17405
Epoch: 9, Train loss: 1136.22175 Test loss : 292.88194
Epoch: 10, Train loss: 1149.49469 Test loss : 269.87857
Epoch: 11, Train loss: 1151.05677 Test loss : 293.91255
Epoch: 12, Train loss: 1119.53765 Test loss : 274.79239
Epoch: 13, Train loss: 1102.96695 Test loss : 277.45624
Epoch: 14, Train loss: 1123.02735 Test loss : 273.58569
Epoch: 15, Train loss: 1118.39870 Test loss : 278.23013
Epoch: 16, Train loss: 1114.44511 Test loss : 268.97942
Epoch: 17, Train loss: 1119.37647 Test loss : 291.22707
Ep

In [291]:
MODEL

CpGDetector(
  (embedding): Embedding(128, 32)
  (lstm): LSTM(32, 64)
  (drop): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

## Model Validation 

### Predict function for a single text/tensor input 

In [292]:
def predict_full_length_seq(lstm_model,input_string,device,type='tensor'):
    set_seed(13)
    alphabet = 'NACGT'
    dna2int = { a: i for a, i in zip(alphabet, range(5))}
    int2dna = { i: a for a, i in zip(alphabet, range(5))}

    if type!='tensor':
        input_string_ints =  np.array([dna2int[i] for i in list(input_string)])
        input_tensor = torch.from_numpy(input_string_ints).int()
    else:
        input_tensor = input_string
    model.eval()
    with torch.no_grad():
        input_tensor =  input_tensor.unsqueeze(0)
        predictions = lstm_model(input_tensor)
        return predictions.item()

### Testing the predict_full_length_seq function for a single test input 

In [293]:
test_sample_full_len_model ="""
NACNCTTCGTGGANCAATNCAATAACGATCNNTAANNNACAGCGCGGANANACGCCATCNCNGTGNGCAGTNCNAATAGATATCCGCTGCCNCAAANCGGNTGTTAAATGACCTTTNTNTNNCCNCNCN
""".replace('\n','')
test_sample_actual_output = count_cpgs(test_sample_full_len_model)
test_sample_full_len_model,test_sample_actual_output
predict_full_length_seq(MODEL,test_sample_full_len_model,'cpu',type='string')

4.699720859527588

### Evaluating full length (128) model on Testing data 

In [298]:
test_predictions_model_1 = []
test_actuals_model_1 = []
for i in zip(test_x,test_y):
    input_tensor,input_target = i
    input_tensor = torch.tensor(input_tensor)
    input_target = torch.tensor(input_target)
    model_prediction = predict_full_length_seq(MODEL,input_tensor,'cpu',type='tensor')
    test_predictions_model_1.append(model_prediction)
    actual = input_target.item()
    test_actuals_model_1.append(actual)

### Mean Absolute Error on Test Data

In [300]:
MAE(test_predictions_model_1,test_actuals_model_1)
# on average there is difference of 1.6 between actual CpG count and predicted CpG count

1.604596383869648

# Part 2: What if the DNA sequences are not the same length

## Data Preparation (Variable Sequence Length)

In [301]:
# DO NOT CHANGE HERE
def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
set_seed(13)

# Use this for getting x label
def rand_sequence(n_seqs: int, seq_len: int = 128) -> Sequence[int]:
    for i in range(n_seqs):
        yield [random.randint(0, 4) for _ in range(seq_len)]


# Use this for getting x label
def rand_sequence_var_len(n_seqs: int, lb: int = 16, ub: int = 128) -> Sequence[int]:
    for i in range(n_seqs):
        seq_len = random.randint(lb, ub)
        yield [random.randint(1, 5) for _ in range(seq_len)]


# Use this for getting y label
def count_cpgs(seq: str) -> int:
    cgs = 0
    for i in range(0, len(seq) - 1):
        dimer = seq[i:i + 2]
        # note that seq is a string, not a list
        if dimer == "CG":
            cgs += 1
    return cgs

In [302]:
# Alphabet helpers
alphabet = 'NACGT'
dna2int = {a: i for a, i in zip(alphabet, range(1, 6))}
int2dna = {i: a for a, i in zip(alphabet, range(1, 6))}
dna2int.update({"pad": 0})
int2dna.update({0: "<pad>"})

intseq_to_dnaseq = partial(map, int2dna.get)
dnaseq_to_intseq = partial(map, dna2int.get)

In [303]:
# TODO complete the task based on the change
def prepare_data(num_samples=100, min_len=16, max_len=128):
    X_dna_seqs_train = list(rand_sequence_var_len(num_samples, min_len, max_len))
    temp =[[int2dna[j] for j in i] for i in X_dna_seqs_train]
    y_dna_seqs =  [count_cpgs(''.join(i)) for i in temp]
    return X_dna_seqs_train, y_dna_seqs

min_len, max_len = 64, 128
train_x, train_y = prepare_data(2048, min_len, max_len)
test_x, test_y = prepare_data(512, min_len, max_len)

In [304]:
# creating the dataset class for data loader 
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, lists, labels): 
        self.lists = lists
        self.labels = labels

    def __getitem__(self, index):
        return torch.LongTensor(self.lists[index]), self.labels[index]

    def __len__(self):
        return len(self.lists)

## LSTM Architecture 

In [305]:
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.drop = nn.Dropout(p=0.5)
        self.fc = nn.Linear(2 * hidden_dim, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        embedded = self.embedding(x)  # adding embedding layer 
        output, _ = self.lstm(embedded) # passing the embedding vector through the lstm unit 
        output = self.drop(output[:, -1, :]) # taking the last hidden state after dropout 
        output = self.fc(self.relu(output))  # activation layer 
        return output

## Model Training

### Setting Hyperparameters

In [306]:
model_parameters = {
    'input_dim': 128,
    'embedding_dim': 32,
    'hidden_dim': 64,
    'num_layers': 1,
    'num_classes': 1
}

In [307]:
model = LSTM(**model_parameters)

In [316]:
training_config = {
    'n_epochs': 50,
    'batch_size': 32,
    'learning_rate': 0.01,
    'criterion': torch.nn.MSELoss()
}

In [309]:
optimizer = torch.optim.Adam(model.parameters(), lr=training_config['learning_rate'])

### Creating Collate function for the data loader

In [310]:
def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_sequences = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
    lengths = torch.LongTensor([len(seq) for seq in sequences])
    packed_sequences = torch.nn.utils.rnn.pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)
    return packed_sequences, torch.tensor(labels)


train_x = [(torch.tensor(seq),torch.tensor(target)) for seq,target in zip(train_x,train_y)]
test_x = [(torch.tensor(seq),torch.tensor(target)) for seq,target in zip(test_x,test_y)]
# creating the torch data loader class for train and test
train_data_loader = torch.utils.data.DataLoader(train_x, batch_size=training_config['batch_size'],collate_fn=collate_fn)
test_data_loader = torch.utils.data.DataLoader(test_x, batch_size=training_config['batch_size'],collate_fn=collate_fn)

In [311]:
print(model)

LSTM(
  (embedding): Embedding(128, 32)
  (lstm): LSTM(32, 64, batch_first=True, bidirectional=True)
  (drop): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)


### Training Loop

In [312]:
def train(model, data_loader, loss_function, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        packed_sequence, targets = batch
        data, lengths = pad_packed_sequence(packed_sequence, batch_first=True)
        data = data.to(device)
        targets = targets.to(device).float()
        targets = targets.unsqueeze(0).T
        
        predictions = model(data)
        loss = loss_function(predictions, targets)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss

In [313]:
def validate(model, data_loader, loss_function, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            packed_sequence, targets = batch
            data, lengths = pad_packed_sequence(packed_sequence, batch_first=True)
            data = data.to(device)
            targets = targets.to(device).float()
            targets = targets.unsqueeze(0).T

            predictions = model(data)
            loss = loss_function(predictions, targets)
            total_loss += loss.item()

    return total_loss

In [314]:
def train_model(model, train_loader, valid_loader, loss_function, optimizer, device, num_epochs):
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, loss_function, optimizer, device)
        valid_loss = validate(model, valid_loader, loss_function, device)
        print(f"Epoch {epoch+1}/{num_epochs} ; Train Loss: {train_loss:.4f} ; Valid Loss: {valid_loss:.4f}")

In [317]:
set_seed(13)
train_model(
    model, 
    train_data_loader, 
    test_data_loader, 
    training_config['criterion'], 
    optimizer, 
    'cpu', 
    training_config['n_epochs'])

Epoch 1/50 ; Train Loss: 237.4198 ; Valid Loss: 64.0410
Epoch 2/50 ; Train Loss: 237.3519 ; Valid Loss: 63.3353
Epoch 3/50 ; Train Loss: 235.7609 ; Valid Loss: 62.4794
Epoch 4/50 ; Train Loss: 237.8076 ; Valid Loss: 63.2557
Epoch 5/50 ; Train Loss: 237.3387 ; Valid Loss: 62.9999
Epoch 6/50 ; Train Loss: 236.7121 ; Valid Loss: 62.6256
Epoch 7/50 ; Train Loss: 235.7550 ; Valid Loss: 62.2630
Epoch 8/50 ; Train Loss: 233.6606 ; Valid Loss: 62.1105
Epoch 9/50 ; Train Loss: 232.7535 ; Valid Loss: 63.1441
Epoch 10/50 ; Train Loss: 232.5397 ; Valid Loss: 62.0602
Epoch 11/50 ; Train Loss: 230.7555 ; Valid Loss: 61.5101
Epoch 12/50 ; Train Loss: 228.1994 ; Valid Loss: 55.9649
Epoch 13/50 ; Train Loss: 220.3994 ; Valid Loss: 58.2307
Epoch 14/50 ; Train Loss: 213.6784 ; Valid Loss: 62.2084
Epoch 15/50 ; Train Loss: 212.1491 ; Valid Loss: 59.7709
Epoch 16/50 ; Train Loss: 223.0662 ; Valid Loss: 61.3179
Epoch 17/50 ; Train Loss: 217.5468 ; Valid Loss: 63.6242
Epoch 18/50 ; Train Loss: 223.5398 ; Val

In [318]:
model

LSTM(
  (embedding): Embedding(128, 32)
  (lstm): LSTM(32, 64, batch_first=True, bidirectional=True)
  (drop): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)

## Model Validation

### Predict Function for variable length input model

In [319]:
def predict(lstm_model,input_string,device,type='tensor'):
    if type!='tensor':
        input_string_ints =  np.array([dna2int[i] for i in list(input_string)])
        input_tensor = torch.from_numpy(input_string_ints).int()
    else:
        input_tensor = input_string
    model.eval()
    with torch.no_grad():
        sequences = input_tensor
        sequences = sequences.unsqueeze(0)
        padded_sequences = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        lengths = torch.LongTensor([len(seq) for seq in sequences])
        packed_sequences = torch.nn.utils.rnn.pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)
        data, lengths = pad_packed_sequence(packed_sequences, batch_first=True)
        data = data.to(device)
        predictions = lstm_model(data)
        return predictions

### Testing the predict function for a string input 

In [323]:
input_string = 'ANNNCGCGCGNGNNACGCGCNNCG'
predict(model,input_string,'cpu',type='string')

tensor([[4.0485]])

### Performance on the Testing Data using Mean Abosolute Error : 512 Samples 

In [324]:
test_predictions = []
test_actuals = []
for i in test_x:
    input_tensor,input_target = i
    model_prediction = predict(model,input_tensor,'cpu',type='tensor').item()
    test_predictions.append(model_prediction)
    actual = input_target.item()
    test_actuals.append(actual)

### Mean Absolute Error on Test Data

In [326]:
MAE(test_predictions,test_actuals) 
# Interpretation : On average the model prediction has the error of 1.81 compared to the actual target. 

1.8137505017220974